In [16]:
# Connexion
import mysql.connector
import sys

import numpy as np 
import pandas as pd

import string
from nltk import *
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') #pour pos_tag

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression

import pickle

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
def connect():
    try:
        cnx = mysql.connector.connect(host="localhost",user="root",password="root",auth_plugin='mysql_native_password',database="sentiment_analysis")
    except mysql.connector.Error as err:
        cnx = False
        print(err)
        sys.exit(1)
    finally:
        return cnx

In [18]:
conn = connect()
conn
cursor = conn.cursor()

In [48]:
def create_dataframe():
    cursor.execute("""  SELECT *
                    FROM clothing_reviews; """)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows,columns=['Index','Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'])
    df = df.set_index('Index')
    df = df.replace('',np.nan,regex = True)
    return df

In [64]:
df= create_dataframe()

In [21]:
df 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
Index,,,,,,,,,,
1,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
2,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
3,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
4,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
5,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
23482,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23483,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23484,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses


In [22]:
any(df['Clothing ID'].duplicated())

True

In [23]:
df[df['Clothing ID']==1104]

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
Index,,,,,,,,,,
53,1104,39,NaN,"Love the color and style, but material snags e...",3,1,3,General,Dresses,Dresses
170,1104,47,Antoher beautiful maeve dress!,I love the dresses by maeve and this one is no...,5,1,3,General,Dresses,Dresses
214,1104,47,My new favorite dress,"I'm 5'4"", 130 lbs. 34 d. i bought the medium. ...",5,1,0,General,Dresses,Dresses
265,1104,25,Amazing peach color!,I ordered this dress in both colors. the peach...,5,1,6,General,Dresses,Dresses
267,1104,54,Perfection.,This dress is incredible. i saw a sales girl w...,5,1,0,General,Dresses,Dresses
...,...,...,...,...,...,...,...,...,...,...
23476,1104,41,Cute dress,Cute dress but not for me. the waist is too h...,3,1,0,General Petite,Dresses,Dresses
23479,1104,32,Unflattering,I was surprised at the positive reviews for th...,1,0,0,General Petite,Dresses,Dresses
23482,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses


In [32]:
df_ori = df.copy()

In [166]:
reviews = df[["Clothing ID","Age","Title","Review Text","Rating","Recommended IND","Positive Feedback Count"]]
reviews

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count
Index,,,,,,,
1,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0
2,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4
3,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0
4,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
5,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6
...,...,...,...,...,...,...,...
23482,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0
23483,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0
23484,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1


In [167]:
clothes = df[["Clothing ID","Division Name","Department Name","Class Name"]]
clothes

,Clothing ID,Division Name,Department Name,Class Name
Index,,,,
1,767,Initmates,Intimate,Intimates
2,1080,General,Dresses,Dresses
3,1077,General,Dresses,Dresses
4,1049,General Petite,Bottoms,Pants
5,847,General,Tops,Blouses
...,...,...,...,...
23482,1104,General Petite,Dresses,Dresses
23483,862,General Petite,Tops,Knits
23484,1104,General Petite,Dresses,Dresses


In [69]:
divisions = df["Division Name"].unique()
divisions = pd.DataFrame(divisions,columns=['Division Name'])
divisions = divisions.dropna()
divisions

,Division Name
0,Initmates
1,General
2,General Petite


In [66]:
departments = df["Department Name"].unique()
departments = pd.DataFrame(departments,columns=['Department Name'])
departments = departments.dropna()
departments

,Department Name
0,Intimate
1,Dresses
2,Bottoms
3,Tops
4,Jackets
5,Trend


In [192]:
classes = df["Class Name"].unique()
classes = [item for item in classes if not(pd.isnull(item)) == True]
classes = pd.DataFrame(classes,columns=['Class Name'])
classes = classes.dropna()
classes

,Class Name
0,Intimates
1,Dresses
2,Pants
3,Blouses
4,Knits
5,Outerwear
6,Lounge
7,Sweaters
8,Skirts
9,Fine gauge


In [168]:
# Applying the condition
clothes.loc[clothes["Division Name"] == 'Initmates', "Division Name"] = 1
clothes.loc[clothes["Division Name"] == 'General', "Division Name"] = 2
clothes.loc[clothes["Division Name"] == 'General Petite', "Division Name"] = 3

In [169]:
# Applying the condition
clothes.loc[clothes["Department Name"] == 'Intimate', "Department Name"] = 1
clothes.loc[clothes["Department Name"] == 'Dresses', "Department Name"] = 2
clothes.loc[clothes["Department Name"] == 'Bottoms', "Department Name"] = 3
clothes.loc[clothes["Department Name"] == 'Tops', "Department Name"] = 4
clothes.loc[clothes["Department Name"] == 'Jackets', "Department Name"] = 5
clothes.loc[clothes["Department Name"] == 'Trend', "Department Name"] = 6

In [170]:
# Applying the condition
clothes.loc[clothes["Class Name"] == 'Intimates', "Class Name"] = 1
clothes.loc[clothes["Class Name"] == 'Dresses', "Class Name"] = 2
clothes.loc[clothes["Class Name"] == 'Pants', "Class Name"] = 3
clothes.loc[clothes["Class Name"] == 'Blouses', "Class Name"] = 4
clothes.loc[clothes["Class Name"] == 'Knits', "Class Name"] = 5
clothes.loc[clothes["Class Name"] == 'Outerwear', "Class Name"] = 6
clothes.loc[clothes["Class Name"] == 'Lounge', "Class Name"] = 7
clothes.loc[clothes["Class Name"] == 'Sweaters', "Class Name"] = 8
clothes.loc[clothes["Class Name"] == 'Skirts', "Class Name"] = 9
clothes.loc[clothes["Class Name"] == 'Fine gauge', "Class Name"] = 10
clothes.loc[clothes["Class Name"] == 'Sleep', "Class Name"] = 11
clothes.loc[clothes["Class Name"] == 'Jackets', "Class Name"] = 12
clothes.loc[clothes["Class Name"] == 'Swim', "Class Name"] = 13
clothes.loc[clothes["Class Name"] == 'Trend', "Class Name"] = 14
clothes.loc[clothes["Class Name"] == 'Jeans', "Class Name"] = 15
clothes.loc[clothes["Class Name"] == 'Legwear', "Class Name"] = 16
clothes.loc[clothes["Class Name"] == 'Shorts', "Class Name"] = 17
clothes.loc[clothes["Class Name"] == 'Layering', "Class Name"] = 18
clothes.loc[clothes["Class Name"] == 'Casual bottoms', "Class Name"] = 19
clothes.loc[clothes["Class Name"] == 'Chemises', "Class Name"] = 20

In [171]:
clothes = clothes.sort_values('Clothing ID').drop_duplicates()

In [87]:
# clothes[clothes.duplicated(subset="Clothing ID")==True]

,Clothing ID,Division Name,Department Name,Class Name
Index,,,,
11714,815,3,4,4
22108,816,3,4,4
2697,817,2,4,4
3823,818,3,4,4
16735,819,3,4,4
...,...,...,...,...
5099,1133,2,5,6
948,1140,3,6,14
5372,1142,2,6,14


In [90]:
# clothes[clothes["Clothing ID"]==815]

,Clothing ID,Division Name,Department Name,Class Name
Index,,,,
18185,815,2,4,4
11714,815,3,4,4


In [103]:
divisions = divisions.reset_index(level=0)
divisions.rename(columns = {'index':'Divisions_ID'}, inplace = True)

In [108]:
departments = departments.reset_index(level=0)
departments.rename(columns = {'index':'Departments_ID'}, inplace = True)

In [193]:
classes = classes.reset_index(level=0)
classes.rename(columns = {'index':'Classes_ID'}, inplace = True)

In [183]:
#Car 0 en id c'est as possible en SQL (index set à 0 !)
divisions["Divisions_ID"] += 1
departments['Departments_ID'] += 1
classes['Classes_ID'] += 1

In [172]:
clothes = clothes.drop_duplicates("Clothing ID", keep='last')
clothes = clothes.set_index('Clothing ID')
clothes = clothes.reset_index(level=0)

In [175]:
reviews = reviews.reset_index(level=0)
reviews.rename(columns = {'Index':'Reviews_ID'}, inplace = True)

In [164]:
# reviews[reviews['Clothing ID']==0]

In [177]:
#Car il y a 1 ID de Clothes à 0 et 0 veut dire NaN en SQL
clothes["Clothing ID"] += 1
reviews['Clothing ID'] += 1

In [184]:
divisions.to_csv(".\datasets\data_clothing_reviews\divisions.csv",index=False)

In [185]:
departments.to_csv(".\datasets\data_clothing_reviews\departments.csv",index=False)

In [195]:
classes.to_csv(".\datasets\data_clothing_reviews\classes.csv",index=False)

In [178]:
clothes.to_csv(".\datasets\data_clothing_reviews\clothes.csv",index=False)

In [180]:
reviews.to_csv("./datasets/data_clothing_reviews/reviews.csv",index=False)

In [181]:
clothes

,Clothing ID,Division Name,Department Name,Class Name
0,1,2,5,6
1,2,1,1,18
2,3,2,4,5
3,4,2,4,8
4,5,2,4,8
...,...,...,...,...
1201,1202,2,2,2
1202,1203,3,2,2
1203,1204,1,1,18
1204,1205,3,3,3


In [182]:
reviews

,Reviews_ID,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count
0,1,768,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0
1,2,1081,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4
2,3,1078,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0
3,4,1050,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
4,5,848,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6
...,...,...,...,...,...,...,...,...
23481,23482,1105,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0
23482,23483,863,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0
23483,23484,1105,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1
23484,23485,1085,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2


In [196]:
classes

,Classes_ID,Class Name
0,1,Intimates
1,2,Dresses
2,3,Pants
3,4,Blouses
4,5,Knits
5,6,Outerwear
6,7,Lounge
7,8,Sweaters
8,9,Skirts
9,10,Fine gauge


In [188]:
departments

,Departments_ID,Department Name
0,1,Intimate
1,2,Dresses
2,3,Bottoms
3,4,Tops
4,5,Jackets
5,6,Trend


In [189]:
divisions

,Divisions_ID,Division Name
0,1,Initmates
1,2,General
2,3,General Petite


In [197]:
def connect():
    try:
        cnx = mysql.connector.connect(host="localhost",user="root",password="root",auth_plugin='mysql_native_password',database="sentiment_analysis_bdd")
    except mysql.connector.Error as err:
        cnx = False
        print(err)
        sys.exit(1)
    finally:
        return cnx

In [198]:
conn = connect()
conn
cursor = conn.cursor()

In [226]:
def create_dataframe2():
    cursor.execute("""  SELECT reviews_id,reviews_clothes_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
                        FROM reviews
                        LEFT JOIN clothes ON reviews.reviews_clothes_id = clothes.clothes_id
                        LEFT JOIN divisions ON clothes.clothes_divisions_id = divisions.divisions_id
                        LEFT JOIN departments ON clothes.clothes_departments_id = departments.departments_id
                        LEFT JOIN classes ON clothes.clothes_classes_id = classes.classes_id """)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows,columns=['Index','Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'])
    df['Division Name'] = df['Division Name'].replace({r'\r': ''}, regex=True)
    df['Department Name'] = df['Department Name'].replace({r'\r': ''}, regex=True)
    df['Class Name'] = df['Class Name'].replace({r'\r': ''}, regex=True)
    df = df.set_index('Index')
    df = df.replace('',np.nan,regex = True)
    return df

In [227]:
df_final = create_dataframe2()

In [228]:
df_final

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
Index,,,,,,,,,,
1,768,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
2,1081,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General Petite,Dresses,Dresses
3,1078,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General Petite,Dresses,Dresses
4,1050,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
5,848,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General Petite,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
23482,1105,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General,Dresses,Dresses
23483,863,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23484,1105,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General,Dresses,Dresses
